# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
#!pip install gmaps
import gmaps
import os

# Import API key
from api_keys import g_key


In [2]:
#!jupyter nbextension enable --py gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
filepath = '../WeatherPy/weather_cities_output.csv'
weather_df = pd.read_csv(filepath)
weather_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Yulara,-25.2406,130.9889,89.44,20,0,16.11,AU,1638933398
1,1,Kasimov,54.9411,41.3953,34.56,100,100,8.93,RU,1638933399
2,2,Hasaki,35.7333,140.8333,50.40,85,100,39.44,JP,1638933400
3,3,Kaitangata,-46.2817,169.8464,64.92,76,98,0.76,NZ,1638933400
4,4,Great Yarmouth,52.6083,1.7305,43.23,75,37,24.61,GB,1638933401
...,...,...,...,...,...,...,...,...,...,...
551,551,Shimoda,34.6667,138.9500,55.76,68,100,10.45,JP,1638933720
552,552,Oskarshamn,57.2645,16.4484,31.98,72,99,13.65,SE,1638933720
553,553,Pegnitz,49.7564,11.5450,32.49,96,96,8.34,DE,1638933721
554,554,Cedar City,37.6775,-113.0619,42.40,71,90,3.44,US,1638933721


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = weather_df["Humidity"]

In [8]:
#Plot heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=93,
                                 point_radius=1)
 

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

#fig not displaying??

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
#max temp lower than 80 degrees but higher than 70
#drop nulls #cloudiness = 0  #windspeed less than 10

ideal_weather_df = weather_df.loc[(weather_df["Max Temp"] >= 70) 
                                        & (weather_df["Max Temp"] <= 80)
                                        & (weather_df["Wind Speed"] <= 10) 
                                        & (weather_df["Cloudiness"]== 0)].dropna()

ideal_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,Sātkhira,22.7167,89.1000,75.09,79,0,7.02,BD,1638933429
97,97,Salalah,17.0151,54.0924,73.49,27,0,8.05,OM,1638933459
199,199,Begusarai,25.4167,86.1333,70.90,49,0,4.74,IN,1638933518
223,223,Santa Fe,-31.6333,-60.7000,77.00,46,0,4.00,AR,1638933534
241,241,Niamey,13.5137,2.1098,71.89,18,0,4.61,NE,1638933544
265,265,Hong Kong,22.2855,114.1577,73.96,58,0,3.00,HK,1638933305
398,398,Maceió,-9.6658,-35.7353,78.24,83,0,4.61,BR,1638933632
405,405,Myanaung,18.2833,95.3167,77.79,68,0,7.76,MM,1638933636
521,521,Beroroha,-21.6667,45.1667,78.73,59,0,4.72,MG,1638933704
527,527,Saint-Pierre,-21.3393,55.4781,78.48,72,0,5.75,RE,1638933707


### Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.


In [35]:
#df contains city ,country, lat, lng, hotel name
hotel_df = ideal_weather_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
47,Sātkhira,BD,22.7167,89.1000,
97,Salalah,OM,17.0151,54.0924,
199,Begusarai,IN,25.4167,86.1333,
223,Santa Fe,AR,-31.6333,-60.7000,
241,Niamey,NE,13.5137,2.1098,
265,Hong Kong,HK,22.2855,114.1577,
398,Maceió,BR,-9.6658,-35.7353,
405,Myanaung,MM,18.2833,95.3167,
521,Beroroha,MG,-21.6667,45.1667,
527,Saint-Pierre,RE,-21.3393,55.4781,


* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [43]:
#Loop through cities
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params["location"]=f"{lat},{lng}"

    #API Request
    #hoteldata = requests.get(base_url, params).json()
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    #print(hoteldata)

    #Exception   
    try:
        hotelname= hoteldata["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"]= hotelname
        print(hotelname)
        
    except:
        hotel_df.loc[index,"Hotel Name"]= np.nan
        print('-----Hotel not found, skipping...')
        
#Drop NaN
hotel_df.dropna(inplace=True)

hotel_df

Retrieving Results for Index 97: Salalah.
Mermaid Beach Resort vengurla
Retrieving Results for Index 199: Begusarai.
Mermaid Beach Resort vengurla
Retrieving Results for Index 223: Santa Fe.
Mermaid Beach Resort vengurla
Retrieving Results for Index 241: Niamey.
Mermaid Beach Resort vengurla
Retrieving Results for Index 265: Hong Kong.
Mermaid Beach Resort vengurla
Retrieving Results for Index 398: Maceió.
Mermaid Beach Resort vengurla
Retrieving Results for Index 405: Myanaung.
Mermaid Beach Resort vengurla
Retrieving Results for Index 527: Saint-Pierre.
Mermaid Beach Resort vengurla
Retrieving Results for Index 547: Vengurla.
Mermaid Beach Resort vengurla


,City,Country,Lat,Lng,Hotel Name
97,Salalah,OM,17.0151,54.0924,Mermaid Beach Resort vengurla
199,Begusarai,IN,25.4167,86.1333,Mermaid Beach Resort vengurla
223,Santa Fe,AR,-31.6333,-60.7000,Mermaid Beach Resort vengurla
241,Niamey,NE,13.5137,2.1098,Mermaid Beach Resort vengurla
265,Hong Kong,HK,22.2855,114.1577,Mermaid Beach Resort vengurla
398,Maceió,BR,-9.6658,-35.7353,Mermaid Beach Resort vengurla
405,Myanaung,MM,18.2833,95.3167,Mermaid Beach Resort vengurla
527,Saint-Pierre,RE,-21.3393,55.4781,Mermaid Beach Resort vengurla
547,Vengurla,IN,15.8667,73.6333,Mermaid Beach Resort vengurla


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display figure
